In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Sprint Race Results

In [4]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_sprint`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [5]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP=GP[GP["Position"]!=0]
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,10,NaT,NaT,NaT,0 days 00:29:44.191000,Finished,0,2021,São Paulo Grand Prix
1,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,13,0,14,NaT,NaT,NaT,0 days 00:28:37.451000,Finished,0,2021,Italian Grand Prix
2,31,E OCON,OCO,OCO,Alpine,0090FF,0,Esteban,Ocon,Esteban Ocon,0,0,10,0,13,NaT,NaT,NaT,0 days 00:26:25.821000,Finished,0,2021,British Grand Prix
3,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,9,0,9,NaT,NaT,NaT,0 days 00:25:20.843000,Finished,0,2023,Belgian Grand Prix
4,31,E OCON,OCO,OCO,Alpine,2293D1,0,Esteban,Ocon,Esteban Ocon,0,0,7,0,8,NaT,NaT,NaT,0 days 00:31:03.332000,Finished,2,2023,Austrian Grand Prix


In [110]:
year=2024

In [111]:
grouped = GP.groupby(['Year','Abbreviation','Position'])["Points"].agg('count').reset_index()
grouped.columns=['Year','Abbreviation','Position','Count']
grouped=grouped[grouped['Year']==year]
grouped=grouped.sort_values(by=['Position','Count'], ascending=[True,False])
grouped.head()

,Year,Abbreviation,Position,Count
254,2024,VER,1,3
216,2024,HAM,2,1
221,2024,LEC,2,1
235,2024,PIA,2,1
233,2024,PER,3,2


In [112]:
fig=px.bar(grouped, 
           x="Position", 
           y='Count', 
           color='Abbreviation',
        #    title="Sprint Race Position Counts By Driver for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5)
)
# fig.update_traces(marker={'size': 12})

In [113]:
fig.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Count by Pos No./{}/Sprint Race Final Position Counts By Driver.html".format(year),full_html=False, include_plotlyjs='cdn')

In [114]:
#By Team

In [115]:
grouped_team = GP.groupby(['Year','TeamName','Position'])["Points"].agg('count').reset_index()
grouped_team.columns=['Year','Constructor','Position','Count']
grouped_team=grouped_team[grouped_team['Year']==year]
grouped_team=grouped_team.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_team.head()

,Year,Constructor,Position,Count
218,2024,Red Bull Racing,1,3
186,2024,Ferrari,2,1
201,2024,McLaren,2,1
206,2024,Mercedes,2,1
219,2024,Red Bull Racing,3,2


In [116]:
fig_team=px.bar(grouped_team, 
           x="Position", 
           y='Count', 
           color='Constructor',
        #    title="Sprint Race Position Counts By Team for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            },
            )

fig_team.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

# fig.update_traces(marker={'size': 12})

In [117]:
fig_team.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Count by Pos No./{}/Sprint Race Final Position Counts By Team.html".format(year),full_html=False, include_plotlyjs='cdn')

In [118]:
#Sprint Qualy

In [119]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_sprint_qualifying`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2_qualy = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [120]:
GP_qualy=df_bq2_qualy
GP_qualy=GP_qualy.fillna(0)
GP_qualy["Position"]=GP_qualy["Position"].astype(int)
GP_qualy["GridPosition"]=GP_qualy["GridPosition"].astype(int)
GP_qualy["Points"]=GP_qualy["Points"].astype(int)
GP_qualy=GP_qualy[GP_qualy["Position"]!=0]
GP_qualy.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,22,Y TSUNODA,TSU,,RB,6692FF,,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,15,,0,0 days 00:01:28.687000,NaT,NaT,NaT,,0,2024,Miami Grand Prix
1,22,Y TSUNODA,TSU,,RB,6692FF,,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,19,,0,0 days 00:01:37.892000,NaT,NaT,NaT,,0,2024,Chinese Grand Prix
2,22,Y TSUNODA,TSU,,RB,6692FF,,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,14,,0,0 days 00:01:06.557000,0 days 00:01:05.960000,NaT,NaT,,0,2024,Austrian Grand Prix
3,3,D RICCIARDO,RIC,,RB,6692FF,,Daniel,Ricciardo,Daniel Ricciardo,https://media.formula1.com/d_driver_fallback_i...,AUS,4,,0,0 days 00:01:28.700000,0 days 00:01:28.122000,0 days 00:01:28.044000,NaT,,0,2024,Miami Grand Prix
4,3,D RICCIARDO,RIC,,RB,6692FF,,Daniel,Ricciardo,Daniel Ricciardo,https://media.formula1.com/d_driver_fallback_i...,AUS,14,,0,0 days 00:01:37.321000,0 days 00:01:36.553000,NaT,NaT,,0,2024,Chinese Grand Prix


In [121]:
grouped_qualy = GP_qualy.groupby(['Year','Abbreviation','Position'])["Points"].agg('count').reset_index()
grouped_qualy.columns=['Year','Abbreviation','Position','Count']
grouped_qualy=grouped_qualy[grouped_qualy['Year']==year]
grouped_qualy=grouped_qualy.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_qualy.head()

,Year,Abbreviation,Position,Count
147,2024,VER,1,2
120,2024,NOR,1,1
108,2024,HAM,2,1
114,2024,LEC,2,1
121,2024,NOR,2,1


In [122]:
fig_qualy=px.bar(grouped_qualy, 
           x="Position", 
           y='Count', 
           color='Abbreviation',
        #    title="Race Position Counts By Driver for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            },
            )

fig_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5)
)
# fig.update_traces(marker={'size': 12})

In [123]:
fig_qualy.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Count by Pos No./{}/Sprint Qualifying Final Position Counts By Driver.html".format(year),full_html=False, include_plotlyjs='cdn')

In [124]:
#By Team

In [125]:
grouped_team_qualy = GP_qualy.groupby(['Year','TeamName','Position'])["Points"].agg('count').reset_index()
grouped_team_qualy.columns=['Year','Constructor','Position','Count']
grouped_team_qualy=grouped_team_qualy[grouped_team_qualy['Year']==year]
grouped_team_qualy=grouped_team_qualy.sort_values(by=['Position','Count'], ascending=[True,False])
grouped_team_qualy.head()

,Year,Constructor,Position,Count
125,2024,Red Bull Racing,1,2
109,2024,McLaren,1,1
94,2024,Ferrari,2,1
110,2024,McLaren,2,1
115,2024,Mercedes,2,1


In [126]:
fig_team_qualy=px.bar(grouped_team_qualy, 
           x="Position", 
           y='Count', 
           color='Constructor',
        #    title="Race Position Counts By Team for the {} F1 Season".format(year),
           height=800, 
           width=1200, 
           template="plotly_white",
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "Alfa Romeo Racing":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
            },
            )

fig_team_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)

# fig.update_traces(marker={'size': 12})

In [127]:
fig_team_qualy.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Count by Pos No./{}/Sprint Qualifying Final Position Counts By Team.html".format(year),full_html=False, include_plotlyjs='cdn')